In [3]:
# Dependencies
import requests
import json
from pprint import pprint
import pandas as pd
import os
import csv
import numpy as np
import time 
import tweepy
from config import consumer_key, consumer_secret, access_token, access_token_secret
from datetime import datetime
import math
import pytz
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


utc=pytz.UTC
startDate = datetime(2018, 1, 1, 0, 0, 0)
endDate = datetime(2018, 4, 10, 0, 0, 0)
startDate = utc.localize(startDate) 
endDate = utc.localize(endDate) 

In [7]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

path2 = "resources/LocalTV_Twitter.csv"
df_TV_raw = pd.read_csv(path2, encoding = "latin1")

df_TV_raw = df_TV_raw[df_TV_raw['TwitterHandle'] != 'None']

print(df_TV_raw.count())
df_TV=df_TV_raw.drop_duplicates(subset= ["TwitterHandle"])
print(df_TV.count())

df_tweet = pd.DataFrame(data=df_TV)
df_tweet.head()

TwitterHandle    589
City             589
StateAbrr        589
dtype: int64
TwitterHandle    578
City             578
StateAbrr        578
dtype: int64


,TwitterHandle,City,StateAbrr
0,@KMSSTV,Shreveport,LA
1,@kvoa,Tucson,AZ
2,@FaceofFox43,Norfolk,VA
3,@thewbdotcom,Burbank,CA
4,@GulfCoastCWCreW,Mobile,AL


In [8]:
# "Real Person" Filters
min_tweets = 5
max_tweets = 500000
max_followers = 20000
max_following = 20000
lang = "en"

# Set other parameters
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify= True, parser=tweepy.parsers.JSONParser())
entry = 0
unique_ids = []
oldest_tweet = None
output_list = pd.DataFrame(columns=['created_at','account_location','text_tweet', 'name'])

In [ ]:
# Super Twitter Loop
for (idx, row) in df_tweet.iterrows():
    #entry += 1 
    user_name = (row.loc['TwitterHandle'])
    # Loop through each user
    counter = 0
    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []
    converted_timestamps = []
    tweet_times = []
    counter_list = [] 
    # Loop through pages.
    for x in range(30):
        # Get all tweets from home feed
        public_tweets = api.user_timeline(user_name, page=x, tweet_mode="extended", count=100)
        #try:
        # Loop through all tweets
        for tweet in public_tweets:
            pprint(tweet)
        #Use filters to check if user meets conditions
            if (tweet['user']["followers_count"] < max_followers and
                tweet['user']["statuses_count"] > min_tweets and
                tweet['user']["statuses_count"] < max_tweets and
                tweet['user']["friends_count"] < max_following and
                tweet["lang"] == lang):
        #Read information
                    created_at= tweet['created_at']
                    created_at =datetime.strptime(created_at, "%a %b %d %H:%M:%S %z %Y")
                    name = tweet['user']['screen_name']                            
                    pprint(created_at)
                    account_location = tweet['user']["location"]
                    pprint(account_location)
                    text_tweet = tweet["full_text"]
                    pprint(text_tweet)
                    output_list.at[idx, 'created_at'] = created_at
                    output_list.at[idx, 'text_tweet'] = text_tweet
                    output_list.at[idx, 'account_location'] = account_location
                    output_list.at[idx, 'name'] = name
#         except Exception as e:
#            print(e)
#            time.sleep(5)
#            print("restarting...")
#            pass

{'contributors': None,
 'coordinates': None,
 'created_at': 'Fri Apr 20 01:00:02 +0000 2018',
 'display_text_range': [0, 116],
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [{'display_url': 'arklatexhomepage.com/news/local-new…',
                        'expanded_url': 'http://www.arklatexhomepage.com/news/local-news/uil-to-use-instant-replay-at-all-texas-hs-football-state-championship-games-starting-in-2018/1130341538?utm_medium=social&utm_source=twitter_KMSSTV',
                        'indices': [93, 116],
                        'url': 'https://t.co/Ijq89P6G4B'}],
              'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'full_text': 'UIL to use instant replay at all Texas HS football state '
              'championship games starting in 2018 https://t.co/Ijq89P6G4B',
 'geo': None,
 'id': 987133757651070976,
 'id_str': '987133757651070976',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status

In [47]:
output_list.head(200)
output_list.to_csv('resources/LocalTV_Tweets.csv')

,created_at,account_location,text_tweet,name
0,2016-05-31 20:49:30+00:00,"Jacksonville, AL",Our last account kept getting hacked so we had...,WLJS_919
1,2018-01-15 17:05:09+00:00,Alabama,You'll hear Danny Glover tomorrow on APR for t...,ALPublicRadio


In [ ]:
output_test = output_list
#.dropna(subset=['place_name_long'])

output_test.count()